In [13]:
#! /usr/bin/python
# -*- coding: utf-8 -*-

import time
import io
import cv2
import numpy
import re
import sqlite3
import dlib
import shutil
import glob
import os
import json


from PIL import Image
from http.server import BaseHTTPRequestHandler, HTTPServer
from os import curdir
from os.path import join as pjoin
from scipy.spatial import distance
from random import choice
from string import ascii_letters
from string import digits
from imutils import face_utils
from datetime import datetime


sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
HOST_NAME = 'localhost' # !!!REMEMBER TO CHANGE THIS!!!
PORT_NUMBER = 1111 # Maybe set this to 9000.
DB_NAME = 'faces.db'


def searchInDb(desc):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    selectedData = []
    sel = cursor.execute("select id, desc, rawstring from face")
    targetIndex = 0
    result = ''
    jsonFilenames = {}
    for line in sel:
        if len(desc) != 0:
            euclidianDistanse = numpy.linalg.norm(numpy.array(line[1].split('|')
                                                              , dtype=float) - numpy.array(desc.split('|'), dtype=float))
            if euclidianDistanse < 0.6:
                targetIndex = line[0]
                result = line[2]
                sel2 = cursor.execute("SELECT images from groupImages WHERE id = ?", (line[0],))
                sel2 = [i[0] for i in sel2]
                jsonFilenames = json.loads(sel2[0])
    
    conn.close()
    if result == "":
        return "<NOTFOUND>"
    else:
        json_dict = {}
        json_dict['content'] = json.loads(result)
        json_dict['id'] = targetIndex
        json_dict['filenames'] = jsonFilenames
        result = json.dumps(json_dict, ensure_ascii=False)
        return result

def create_filename():
    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    sel_filenames = cursor.execute("SELECT images FROM groupImages")
    sel = [i[0] for i in sel_filenames]
    filenames_list_from_json = []

    if len(sel) != 0:
        for elem in sel:
            json_dict = json.loads(elem)
            filenames_list_from_json.append([i for i in json_dict.keys()])

        filenames_list = []
        for elem in filenames_list_from_json:
            for filename in elem:
                filenames_list.append(filename)

        new_filename = "".join(choice(ascii_letters + digits) for i in range(20)) + ".png"
        while filenames_list.count(new_filename) > 1:
            new_filename = "".join(choice(ascii_letters + digits) for i in range(20)) + ".png"
        conn.close()
        return new_filename
    else:
        conn.close()
        return "".join(choice(ascii_letters + digits) for i in range(20)) + ".png"
    
    

def appendToDb(keywords, rawstring, image):
    desc = getDescriptor(image)
    if desc != '':
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        filename = create_filename()
        
        try:        
            ins = cursor.execute("INSERT INTO face (rawstring, keywords, desc, key) VALUES( ?, ?, ?, ?)",
                                 (rawstring, keywords, desc, filename))
            conn.commit()
            sel = cursor.execute("SELECT id FROM face WHERE key = ?", (filename,))
            sel = [i[0] for i in sel]
            jsonDict = {filename: filename}
            jsonString = json.dumps(jsonDict, ensure_ascii=False)
            ins2 = cursor.execute("INSERT INTO groupImages (id, images) VALUES(?, ?)", (sel[0], jsonString))
            conn.commit()
            image_name = 'images\\%s' % filename 
            cv2.imwrite(image_name, image)
                
        except sqlite3.IntegrityError:
            print('Integrity error')
        finally:
            conn.commit()
            conn.close()
        

def getDescriptor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    detector = dlib.get_frontal_face_detector()
    dets = detector(gray, 1)
    faceDescriptor = []
    if len(dets) < 2:
        for i, d in enumerate(dets):
                shape = sp(img, d)
                faceDescriptor.append(str(facerec.compute_face_descriptor(img, shape)).replace('\n', '|'))
        if len(faceDescriptor):
            return faceDescriptor[0]
        else:
            return ""
    else:
        return ""

def parseDataFromPost(dict_of_data):
    keywords = ''
    for value in dict_of_data.values():
        keywords += value + "<key>"
    return keywords

def countOfRowsInDB():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    count = 0
    res = cursor.execute("SELECT COUNT(*) FROM face")
    for line in res:
        count = line
    conn.close()
    return count[0]

def textSearchingInDB(searchString):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    results = []
    res = cursor.execute("SELECT id, keywords FROM face")
    full_select = [line[::1] for line in res]
    conn.close()
    for line in full_select:
        if re.search(searchString.upper(), line[1].upper()):
            results.append(line[0])
    if len(results) == 0:
        return "NOTFOUND"
    else:
        return(results)

def prepare_searched_result(searchString):
    searched_results = textSearchingInDB(searchString)
    result = []
    images_names = []
    ids = []
    if searched_results != "NOTFOUND":
        for line in searched_results:
            conn = sqlite3.connect(DB_NAME)
            cursor = conn.cursor()
            sel = cursor.execute("select rawstring, key, id from face where id=?", [line])
            sel = [i for i in sel]
            result.append([i[0] for i in sel])
            images_names.append([i[1] for i in sel])
            ids.append([i[2] for i in sel])
            conn.close()
        return result, images_names, ids
    else:
        return "", "", ""

def prepare_searched_response(searchString):
    string, image_names, ids = prepare_searched_result(searchString)
    json_dict = {}
    
    if len(string) != 0 and len(image_names) != 0 and len(ids) !=0:
        for i, line in enumerate(image_names):
            json_dict.update({'record' + str(i) : {"id":ids[i][0]}})
            json_dict['record' + str(i)].update({'filename': image_names[i][0]})
            json_dict['record' + str(i)].update({'record_content': json.loads(string[i][0])})
        
        result = json.dumps(json_dict, ensure_ascii=False)
        return result.encode()
    else:
        return "NOTFOUND".encode()

def delete_record(request_string):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    sel = cursor.execute("select images from groupImages where id=?", (request_string,))
    filename = ([i[0] for i in sel])
    filename = json.loads(filename[0])
    filename = [i for i in filename.keys()]
    delete = cursor.execute("DELETE FROM face WHERE id=?", (request_string,))
    delete = cursor.execute("DELETE FROM groupImages WHERE id=?", (request_string,))
    conn.commit()
    conn.close()
    for i in filename:
        os.remove('images\\' + i)
    return '<OK>'

def update_record(request_string):
#     Добавить строку с 
    id_in_db = request_string['id']
    keywords = parseDataFromPost(request_string['content'])
    json_string = json.dumps(request_string['content'], ensure_ascii=False)
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    upd = cursor.execute("UPDATE face SET rawstring = ?, keywords = ? WHERE id = ?",
                         (json_string, keywords, id_in_db))
    conn.commit()
    conn.close()
    return "<OK>"

def add_new_image(id_in_db, image):
    desc = getDescriptor(image)
    if desc != "":
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        sel = cursor.execute("select images from groupImages where id=?", (id_in_db, ))
        json_str = ([i[0] for i in sel])
        json_dictionary = json.loads(json_str[0])
        names = [i for i in json_dictionary.keys()]
#         print(names)
        filename = create_filename()
        
        while names.count(filename) > 1:
                    filename = create_filename()
        
        json_dictionary.update({filename : filename})
        image_name = 'images\\%s' % filename 
        cv2.imwrite(image_name, image)
#         print(json_dictionary)
        json_str = json.dumps(json_dictionary, ensure_ascii=False)
        
        try:
            cursor.execute("UPDATE face SET desc = ? WHERE id = ?", (desc, id_in_db))
            cursor.execute("UPDATE groupImages SET images = ? WHERE id = ?", (json_str, id_in_db))
            return "<UPDATED>"
        except sqlite3.IntegrityError:
            print('Integrity error')
            return "<NOT UPDATED>"
        finally:
            conn.commit()
            conn.close()
    else:
        return "<NOT UPDATED>"
    
def update_user_record_or_create_new(image, info_dict):
    
    keywords = parseDataFromPost(info_dict)
    user_information_json = json.dumps(info_dict, ensure_ascii=False)
    
    face_desc = getDescriptor(image)
    if face_desc != "":
        
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        sel_id_and_desc = cursor.execute("SELECT id, desc FROM face")
        sel_id_and_desc = [i for i in sel_id_and_desc]
        added = False
        filename = create_filename()
        if len(sel_id_and_desc) != 0:
            for elem in sel_id_and_desc:
                filename = create_filename()
                euclidian_distance = numpy.linalg.norm(numpy.array(elem[1].split('|'), dtype=float) - numpy.array(face_desc.split('|'), dtype=float))
                if euclidian_distance < 0.6:
                    added = True
                    sel_image_names = cursor.execute("SELECT images FROM groupImages WHERE id = ?", (elem[0], ))
                    sel_image_names = [i[0] for i in sel_image_names]
                    image_names_dict = json.loads(sel_image_names[0])
                    
                    image_names_dict.update({filename:filename})
                    image_names_json_string = json.dumps(image_names_dict, ensure_ascii=False)
                    
                    upd = cursor.execute("UPDATE face SET desc = ?, rawstring = ?, keywords = ?, key = ? WHERE id = ?", 
                                        (face_desc, user_information_json, keywords, filename, elem[0]))
                    conn.commit()
                    upd2 = cursor.execute("UPDATE groupImages SET images = ? WHERE id = ?", (image_names_json_string, elem[0]))
                    conn.commit()
                    image_name = 'images\\%s' % filename 
                    cv2.imwrite(image_name, image)
                
                
        if added == False or len(sel_id_and_desc) == 0:
            ins = cursor.execute("INSERT INTO face (rawstring, keywords, desc, key) VALUES( ?, ?, ?, ?)",
                             (user_information_json, keywords, face_desc, filename))
            conn.commit()
            sel = cursor.execute("SELECT id FROM face WHERE key = ?", (filename,))
            sel = [i[0] for i in sel]
            images_name_json_dict = {filename: filename}
            images_name_json_string = json.dumps(images_name_json_dict, ensure_ascii=False)
            ins2 = cursor.execute("INSERT INTO groupImages (id, images) VALUES(?, ?)", (sel[0], images_name_json_string))
            conn.commit()
            image_name = 'images\\%s' % filename 
            cv2.imwrite(image_name, image)
        
        conn.commit()
        conn.close()
        
        return "<OK>"
    else:
        return "<BAD DESCRIPTOR>"

    
class MyHandler(BaseHTTPRequestHandler):
#     store_path = pjoin(curdir, '1.jpg')
    
    def _set_headers(self):
        
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        
        self._set_headers()
        if self.path == "/get_users_count":
            countRows = countOfRowsInDB()
            responseString = str(countRows)
            self.wfile.write(responseString.encode())
        
        if re.findall("download_image/images/\w+\.\w{3,4}" ,self.path):
            filename = re.findall("images/\w+\.\w{3,4}", self.path)[0]
            filename.replace("/", "\\")
            f = open(filename, "rb")
            byte = f.read()
            f.close()
            
            self.wfile.write(byte)    

    def do_HEAD(self):
        self._set_headers()
        
    def do_POST(self):
        
        # Doesn't do anything with posted data
        self._set_headers()
        params = self.rfile.read(int(self.headers['Content-Length']))
          
        
        if self.path == '/append_user_to_db':
            paramsSplit = params.split(b'<image>')
            current_date = datetime.now().strftime("%Y.%m.%d %H:%M:%S")
            data_string = paramsSplit[0].decode()
            data_dict = json.loads(data_string)
            data_dict.update({'Дата регистрации':current_date})
            keywords = parseDataFromPost(data_dict)
            img = Image.open(io.BytesIO(paramsSplit[1]))
            img = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2RGB)
            appendToDb(keywords, json.dumps(data_dict, ensure_ascii=False), img)
            self.wfile.write("Succsess POST".encode())
            
        elif self.path == '/identify_and_search_user':
            img = Image.open(io.BytesIO(params))
            img = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2RGB)
            desc = getDescriptor(img)
            searched = searchInDb(desc)
            
            if not searched == "<NOTFOUND>":
                sendingString = "Succsess POST<content>" + searched
                self.wfile.write(sendingString.encode('utf-8'))
            else:
                self.wfile.write("NOTFOUND<content>".encode('utf-8'))

        
        elif self.path == '/full_text_search':
            searchString = str(params.decode())
            result = prepare_searched_response(searchString)
            if not result == b"NOTFOUND":
                self.wfile.write(result)
            else:
                self.wfile.write(result)
                
                
        elif self.path == '/delete_user_record':
            returned_string = delete_record(str(params.decode()))
            self.wfile.write(returned_string.encode())
            
            
        elif self.path == '/update_user_record':
            json_dict = json.loads(str(params.decode()))
            if 'Дата обновления' in json_dict['content']:
                json_dict['content']['Дата обновления'] = datetime.now().strftime("%Y.%m.%d %H:%M:%S")
            else:
                json_dict['content'].update({'Дата обновления' : datetime.now().strftime("%Y.%m.%d %H:%M:%S")})
            returned_string = update_record(json_dict)
            self.wfile.write(returned_string.encode())
        
        elif self.path == '/add_new_image':
            params_split = params.split(b'<image>')
            id_in_db = params_split[0].decode()
            image = Image.open(io.BytesIO(params_split[1]))
            image = cv2.cvtColor(numpy.array(image), cv2.COLOR_BGR2RGB)
            response = add_new_image(id_in_db, image)
            self.wfile.write(response.encode())
            
        elif self.path == '/update_user_record_or_add_new':
            current_date = datetime.now().strftime("%Y.%m.%d %H:%M:%S")
            params_split = params.split(b'<image>')
            image = Image.open(io.BytesIO(params_split[1]))
            image = cv2.cvtColor(numpy.array(image), cv2.COLOR_BGR2RGB)
            info_json_string = params_split[0].decode()
            info_dict = json.loads(info_json_string)
            info_dict.update({'Дата регистрации':current_date})
            reply = update_user_record_or_create_new(image, info_dict)
            self.wfile.write(reply.encode())
            
            
if __name__ == '__main__':
    server_class = HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime(), "Server Starts - %s:%s" % (HOST_NAME, PORT_NUMBER))
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    print(time.asctime(), "Server Stops - %s:%s" % (HOST_NAME, PORT_NUMBER))

Fri Jun  8 22:56:12 2018 Server Starts - localhost:1111


127.0.0.1 - - [08/Jun/2018 22:56:20] "POST /update_user_record_or_add_new HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 22:56:49] "POST /update_user_record_or_add_new HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 22:57:18] "POST /update_user_record_or_add_new HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 22:57:57] "POST /update_user_record_or_add_new HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 22:58:15] "POST /update_user_record_or_add_new HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 22:58:43] "POST /update_user_record_or_add_new HTTP/1.1" 200 -


ААААААЙ МЛЯЯЯ158


127.0.0.1 - - [08/Jun/2018 22:59:00] "POST /update_user_record_or_add_new HTTP/1.1" 200 -


ААААААЙ МЛЯЯЯ162


...

ААААААЙ МЛЯЯЯ158


127.0.0.1 - - [08/Jun/2018 23:01:15] "POST /update_user_record_or_add_new HTTP/1.1" 200 -


ААААААЙ МЛЯЯЯ158


127.0.0.1 - - [08/Jun/2018 23:01:21] "POST /identify_and_search_user HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:23] "GET /download_image/images/CZonkdqVu37P6y7SZVcg.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:23] "GET /download_image/images/kmgDIOLmtqFEd4HtBBlp.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:23] "GET /download_image/images/0p5dZCDVXeQOjp0ESxrD.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:23] "GET /download_image/images/cmW16LMRMbi7JlKuJIcI.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:30] "GET /get_users_count HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:34] "POST /full_text_search HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:34] "GET /download_image/images/cmW16LMRMbi7JlKuJIcI.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:34] "GET /download_image/images/A9bJCos4Vgja6MleSgiS.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2018 23:01:34] "GET /download_image/images/AhjzKaKGWyFX4bL9p4rc.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun

Fri Jun  8 23:02:15 2018 Server Stops - localhost:1111
